In [31]:
import requests
import os
import re
import zipfile

# Replace these with your SoundCloud app credentials
CLIENT_ID = "BIyat3kaxbSqtFEe7vNz0SnTV3RQWAh0"
CLIENT_SECRET = "e17Pefo7Umbie2frb70weYvRR2tmvonV"
REDIRECT_URI = "https://soundcloud.com/tobi-fadeyi-12472812"  # Set this in your app's settings
ACCESS_TOKEN = None  # Will store the OAuth token

def authenticate():
    """Authenticate with SoundCloud and get an access token."""
    global ACCESS_TOKEN
    auth_url = "https://api.soundcloud.com/oauth2/token"
    payload = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials",
    }

    response = requests.post(auth_url, data=payload)

    if response.status_code == 200:
        ACCESS_TOKEN = response.json().get("access_token")
        print("Authenticated successfully!")
    else:
        print(f"Failed to authenticate: {response.status_code}, {response.text}")

def get_playlist_metadata(playlist_url):
    """Fetch playlist metadata from SoundCloud using Authorization header."""
    if not ACCESS_TOKEN:
        print("Authenticate first!")
        return None

    resolve_url = "https://api.soundcloud.com/resolve"
    headers = {
        "Authorization": f"OAuth {ACCESS_TOKEN}"
    }
    params = {
        "url": playlist_url
    }
    response = requests.get(resolve_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching playlist: {response.status_code}, {response.text}")
        return None

def download_tracks(playlist_metadata, save_folder="downloads", zip_filename="playlist.zip"):
    """Download tracks from the playlist and save them into a ZIP file."""
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Create a ZIP file
    zip_path = os.path.join(save_folder, zip_filename)
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for track in playlist_metadata.get('tracks', []):
            track_url = track.get('stream_url')
            track_title = track.get('title')

            if track_url:
                headers = {
                    "Authorization": f"OAuth {ACCESS_TOKEN}"
                }
                response = requests.get(track_url, headers=headers, stream=True)

                if response.status_code == 200:
                    # Sanitize file name
                    safe_title = re.sub(r'[<>:"/\\|?*]', '_', track_title)
                    file_path = os.path.join(save_folder, f"{safe_title}.mp3")

                    # Save track to file and add it to the ZIP archive
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(chunk_size=1024):
                            file.write(chunk)

                    # Add the track to the ZIP file
                    zipf.write(file_path, arcname=f"{safe_title}.mp3")
                    os.remove(file_path)  # Optionally remove the file after adding to the ZIP
                    print(f"Downloaded and added to ZIP: {track_title}")
                else:
                    print(f"Failed to download {track_title}: {response.status_code}")

    print(f"All tracks downloaded and saved to: {zip_path}")

if __name__ == "__main__":
    playlist_url = input("Enter SoundCloud playlist URL: ")
    authenticate()
    playlist_metadata = get_playlist_metadata(playlist_url)

    if playlist_metadata:
        download_tracks(playlist_metadata)


Enter SoundCloud playlist URL: https://soundcloud.com/jayden-obis/sets/jersey-club?si=fa1c5f1d3e82408ca5f2228dceb67139&utm_source=clipboard&utm_medium=text&utm_campaign=social_sharing
Authenticated successfully!
Downloaded and added to ZIP: Snooze (Jersey Club) [B Goodie & Mvntana]
Downloaded and added to ZIP: Want U (Jersey Club - shonci x fazobeats)
Downloaded and added to ZIP: WE COULDA BEEN
Downloaded and added to ZIP: In This Darkness (Jersey Club + Brazilian Funk)
Downloaded and added to ZIP: i must apologise jersey club X black clover
Downloaded and added to ZIP: The Beach GIVEON (JERSEY CLUB)
Downloaded and added to ZIP: no idea X jersey club (slowed + reverb)
Downloaded and added to ZIP: yo ahh tweakin jigsaw (jersey club)
Downloaded and added to ZIP: KilSoSouth - Fallin' Out Jersey Club
Downloaded and added to ZIP: Lil Wayne Feat Drake x Future Love Me Jersey Remix (Prod LordParadize)
Downloaded and added to ZIP: Nicki Minaj - You Already Know (Jersey Club Remix) (prod. by Al

In [33]:
from google.colab import files

In [34]:
files.download('downloads/playlist.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>